In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
#Loading the data
today = str(date.today())
df_branch_service = pd.read_json("branch_service_transaction_info.json")
df_customer_transaction = pd.read_json("customer_transaction_info.json")

In [3]:
#Profiling the data
df_branch_service.head(10)

,txn_id,branch_name,service,price
0,TXN-24546,MallOfAsia,Manicure,NaN
1,TXN-14642,Starmall,HairColor,NaN
2,TXN-60295,SmallMall,FootSpa,NaN
3,TXN-60295,Starmall,FootSpa,NaN
4,TXN-60295,MayMall,FootSpa,NaN
5,TXN-60295,FrankMall,FootSpa,NaN
6,TXN-40462,Starmall,HairColor,NaN
7,TXN-40462,Megamall,HairColor,NaN
8,TXN-08102,RobinsonsMall,HairColor,NaN
9,TXN-08102,SmallMall,HairColor,NaN


In [4]:
df_customer_transaction.head(10)

,txn_id,avail_date,last_name,first_name,birthday
0,TXN-24546,2030-09-08,ORTIZ,EDUARDO,1990-07-08
1,TXN-14642,2026-05-26,NIENOW,LEA,2000-11-26
2,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
3,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
4,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
5,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
6,TXN-40462,2021-08-21,KUHN,TOD,2002-11-25
7,TXN-40462,2021-08-21,KUHN,TOD,2002-11-25
8,TXN-08102,2010-04-03,JOHNSON,MILTON,2003-07-10
9,TXN-08102,2010-04-03,JOHNSON,MILTON,2003-07-10


In [5]:
print(df_branch_service.shape)
print(df_customer_transaction.shape)

(130653, 4)
(130653, 5)


In [6]:
print(df_branch_service['txn_id'].nunique())
print(df_customer_transaction['txn_id'].nunique())

62354
62354


In [7]:
#Dropping duplicates
df_branch_service = df_branch_service.drop_duplicates(subset=['txn_id'])
df_customer_transaction = df_customer_transaction.drop_duplicates(subset=['txn_id'])

In [8]:
#confirming shape
print(df_branch_service.shape)
print(df_customer_transaction.shape)

(62354, 4)
(62354, 5)


In [9]:
#checking null values count per column
df_branch_service.isnull().sum()

txn_id             0
branch_name     8880
service            0
price          11082
dtype: int64

In [10]:
df_customer_transaction.isnull().sum()

txn_id        0
avail_date    0
last_name     0
first_name    0
birthday      0
dtype: int64

In [11]:
#fill up null values with forward fill for branch_name
df_branch_service['branch_name'].unique()

array(['MallOfAsia', 'Starmall', 'SmallMall', 'RobinsonsMall', '', None,
       'MayMall', 'FrankMall', 'N/A', 'Megamall'], dtype=object)

In [12]:
#'' and None are empty values, fill them with forward fill
# first, let '' be null
df_branch_service['branch_name'] = df_branch_service.replace('',np.nan).groupby('txn_id')['branch_name'].transform('first')
# then fill null, use forward and backward to fill
df_branch_service['branch_name'] = df_branch_service['branch_name'].ffill().bfill()

In [13]:
#confirming branch_name has no null
df_branch_service.isnull().sum()

txn_id             0
branch_name        0
service            0
price          11082
dtype: int64

In [14]:
df_branch_service['branch_name'].unique()

array(['MallOfAsia', 'Starmall', 'SmallMall', 'RobinsonsMall', 'MayMall',
       'FrankMall', 'N/A', 'Megamall'], dtype=object)

In [15]:
# group mean fill prices
df_branch_service['price'] = df_branch_service['price'].fillna(df_branch_service.groupby(['branch_name','service'])['price'].transform('mean'))

In [16]:
#confirming price has no null
df_branch_service.isnull().sum()

txn_id         0
branch_name    0
service        0
price          0
dtype: int64

In [17]:
df_branch_service.head(10)

,txn_id,branch_name,service,price
0,TXN-24546,MallOfAsia,Manicure,42.651993
1,TXN-14642,Starmall,HairColor,69.583267
2,TXN-60295,SmallMall,FootSpa,80.037329
6,TXN-40462,Starmall,HairColor,69.583267
8,TXN-08102,RobinsonsMall,HairColor,66.735226
12,TXN-64262,SmallMall,Rebond,312.646002
15,TXN-41934,RobinsonsMall,HairColor,66.735226
19,TXN-39050,Starmall,Manicure,43.364281
21,TXN-43150,Starmall,Haircut,51.769018
22,TXN-49423,RobinsonsMall,Rebond,302.828204


In [18]:
df_merged = pd.merge(df_customer_transaction, df_branch_service)

In [19]:
#profiling of merged dataframe
df_merged.isnull().sum()

txn_id         0
avail_date     0
last_name      0
first_name     0
birthday       0
branch_name    0
service        0
price          0
dtype: int64

In [20]:
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
13479,TXN-36011,2028-08-03,gusikowski,corrine,1990-11-28,MallOfAsia,Haircut,0.000000
13584,TXN-63723,2022-12-07,brekke,leila,1999-04-11,Megamall,NailColor,0.000000
28706,TXN-17624,2028-10-02,Trantow,Miracle,1995-03-05,Starmall,Manicure,55.232400
60917,TXN-48933,2005-10-07,Jewess,Gust,2006-03-24,N/A,NailColor,30.123790
27501,TXN-42381,2021-10-20,McLaughlin,Ernesto,2009-03-01,RobinsonsMall,Rebond,400.231230
40833,TXN-55187,2009-08-25,"Nolan,,,,,,,,",Astrid........,2004-12-18,Starmall,Rebond,400.231230
10219,TXN-30622,2019-09-22,WISOZK,JAVON,1990-07-11,Megamall,Haircut,50.644998
10075,TXN-03505,2027-06-28,RODRIGUEZ,MARCOS,2000-07-24,MallOfAsia,HairColor,68.352993
18801,TXN-23776,2015-09-28,reinger,xavier,2010-08-23,SmallMall,Pedicure,0.000000
15907,TXN-35766,2027-02-22,ryan,zechariah,2002-09-30,RobinsonsMall,NailColor,0.000000


In [21]:
#filter alphabet only in last_name and first_name
df_merged['last_name'] = df_merged['last_name'].str.replace('\W', '', regex=True)
df_merged['first_name'] = df_merged['first_name'].str.replace('\W', '', regex=True)

In [22]:
#uppercase for last_name and first_name
df_merged['last_name'] = df_merged['last_name'].str.upper()
df_merged['first_name'] = df_merged['first_name'].str.upper()

In [23]:
#check if no special characters and all uppercase for names
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
7902,TXN-18174,2018-03-02,CASPER,ALFORD,1992-10-13,Megamall,HairColor,69.964710
502,TXN-49585,2011-06-18,HEANEY,ADOLF,1990-07-27,RobinsonsMall,HairColor,66.735226
14467,TXN-34070,2014-07-10,ERNSER,GERARD,1996-08-26,N/A,Manicure,0.000000
13683,TXN-13921,2020-03-31,KOZEY,JULIANNE,2008-11-25,MayMall,Haircut,0.000000
49837,TXN-49670,2015-04-01,FAHEY,JOANIE,2010-04-21,SmallMall,HairColor,88.093930
60195,TXN-00983,2019-05-07,TURNER,LAURA,1999-07-07,RobinsonsMall,Haircut,66.123457
25082,TXN-25414,2009-04-20,JOHNSTON,ARIEL,2001-08-23,Megamall,Haircut,66.123457
5910,TXN-44415,2024-01-09,DAVIS,REGINALD,1994-01-25,SmallMall,Manicure,43.522917
58076,TXN-55236,2006-02-27,WELCH,EPHRAIM,1999-08-16,N/A,NailColor,30.123790
61292,TXN-42353,2024-10-28,CRIST,KENNY,1997-09-30,SmallMall,HairColor,88.093930


In [24]:
#profiling date columns
print(df_merged['birthday'].min())
print(df_merged['birthday'].max())

print(df_merged['avail_date'].min())
print(df_merged['avail_date'].max())

print(df_merged['avail_date'].describe)
print(df_merged['birthday'].describe)

print(df_merged[(df_merged['avail_date'] <= df_merged['birthday'])])

1990-01-01
2010-12-30
2005-01-01
2030-12-30
<bound method NDFrame.describe of 0        2030-09-08
1        2026-05-26
2        2006-09-25
3        2021-08-21
4        2010-04-03
            ...    
62349    2022-11-21
62350    2020-04-14
62351    2030-01-26
62352    2021-02-27
62353    2025-01-05
Name: avail_date, Length: 62354, dtype: object>
<bound method NDFrame.describe of 0        1990-07-08
1        2000-11-26
2        1993-05-22
3        2002-11-25
4        2003-07-10
            ...    
62349    1997-03-06
62350    1996-05-28
62351    1996-05-28
62352    1995-03-29
62353    1993-01-10
Name: birthday, Length: 62354, dtype: object>
          txn_id  avail_date  last_name first_name    birthday    branch_name  \
9      TXN-49423  2008-08-13      BERGE    RASHEED  2010-02-05  RobinsonsMall   
17     TXN-35244  2005-11-23     HAMMES     EUNICE  2006-06-13      SmallMall   
97     TXN-23898  2005-05-28   BOTSFORD     GIANNI  2009-03-30  RobinsonsMall   
101    TXN-17974  2008-04-12  

In [25]:
#data type of birthday and avail_date should be datetime instead of object
df_merged['avail_date'] = pd.to_datetime(df_merged['avail_date'], format='%Y-%m-%d')
df_merged['birthday'] = pd.to_datetime(df_merged['birthday'], format='%Y-%m-%d')

#confirming
print(df_merged['avail_date'].describe)
print(df_merged['birthday'].describe)

<bound method NDFrame.describe of 0       2030-09-08
1       2026-05-26
2       2006-09-25
3       2021-08-21
4       2010-04-03
           ...    
62349   2022-11-21
62350   2020-04-14
62351   2030-01-26
62352   2021-02-27
62353   2025-01-05
Name: avail_date, Length: 62354, dtype: datetime64[ns]>
<bound method NDFrame.describe of 0       1990-07-08
1       2000-11-26
2       1993-05-22
3       2002-11-25
4       2003-07-10
           ...    
62349   1997-03-06
62350   1996-05-28
62351   1996-05-28
62352   1995-03-29
62353   1993-01-10
Name: birthday, Length: 62354, dtype: datetime64[ns]>


In [26]:
#removing rows with later date than current date
df_merged = df_merged[(df_merged['avail_date'] <= today) & (df_merged['birthday'] <= today)]

In [27]:
#removing rows when avail_date happens before birthday
df_merged = df_merged[(df_merged['avail_date'] > df_merged['birthday'])]

In [28]:
#confirming
print(df_merged['birthday'].max())
print(df_merged['avail_date'].max())

print(df_merged[(df_merged['avail_date'] <= df_merged['birthday'])])

2010-12-30 00:00:00
2023-11-22 00:00:00
Empty DataFrame
Columns: [txn_id, avail_date, last_name, first_name, birthday, branch_name, service, price]
Index: []


In [29]:
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
20024,TXN-29224,2020-01-02,WEST,ASTRID,2008-10-04,RobinsonsMall,Rebond,0.000000
43609,TXN-29676,2011-03-18,TILLMAN,CAMILLE,1997-10-04,Starmall,Pedicure,77.987989
32535,TXN-02423,2006-05-26,ABSHIRE,KAYLIN,2002-06-17,FrankMall,Haircut,66.123457
49256,TXN-17202,2019-10-14,GLEICHNER,KOLE,1994-07-15,Megamall,NailColor,30.123790
34110,TXN-64624,2006-08-17,LEGROS,ERNESTINA,1998-02-05,Megamall,NailColor,30.123790
10585,TXN-18205,2014-07-24,CRUICKSHANK,JESUS,2005-07-16,MayMall,Haircut,53.354789
7197,TXN-58210,2005-05-07,LANGOSH,KYLE,1998-02-03,MallOfAsia,Haircut,53.139701
9096,TXN-24223,2019-04-20,RUNTE,HELGA,2001-01-21,Megamall,Pedicure,63.723784
11016,TXN-30800,2005-04-23,ROGAHN,DELLA,1997-09-08,FrankMall,NailColor,23.338226
56236,TXN-54785,2013-01-06,BOGAN,DORIS,2002-02-12,SmallMall,FootSpa,100.121230


In [30]:
df_merged.shape

(43131, 8)